In [1]:
import pandas as pd
import numpy as np
import os
if os.path.basename(os.getcwd()) == "Belfort":
    os.chdir('..')  ## to work at project root  like in any IDE
    os.chdir('..')

# General assumptions

In [2]:
r=0.04# discount rate
# Carbon tax
d_carbon_tax={2030:120,2040:170,2050:200,2060:200}

# 1. Technologies

In [3]:
Input_folder="Data/input/Prod_model/"
Technos_df=pd.read_csv(Input_folder+"Technos.csv",sep=";",decimal=",").set_index("TECHNOLOGIES")
Technos_df

,CAPEX_2030,CAPEX_2040,CAPEX_2050,CAPEX_2060,Decommissioning_2030,Decommissioning_2040,Decommissioning_2050,Decommissioning_2060,Lifetime_2030,Lifetime_2040,...,minCapacity_100_enr_2040,maxCapacity_100_enr_2040,minCapacity_100_enr_2050,maxCapacity_100_enr_2050,minCapacity_100_enr_2060,maxCapacity_100_enr_2060,RampConstraintPlus,RampConstraintMoins,RampConstraintPlus2,RampConstraintMoins2
TECHNOLOGIES,,,,,,,,,,,,,,,,,,,,,
CCG,8.300000e+05,830000.000,830000.000,830000.000,0,0,0,0,30,30,...,0,12664,0,0,0,0,0.50,0.50,0,0
TAC,4.500000e+05,450000.000,450000.000,450000.000,0,0,0,0,30,30,...,0,0,0,0,0,0,0.00,0.00,0,0
OldNuke,6.500000e+05,650000.000,650000.000,650000.000,320000,320000,320000,320000,10,10,...,42230,42230,13870,13870,0,0,0.04,0.04,0,0
NewNuke,1.157576e+07,5000000.000,4500000.000,4500000.000,200000,200000,200000,200000,60,60,...,1650,1650,1650,1650,0,0,0.04,0.04,0,0
WindOnShore,1.200000e+06,1050000.000,900000.000,900000.000,0,0,0,0,25,30,...,30200,55217,37000,75217,37000,75217,0.00,0.00,0,0
WindOffShore,2.036735e+06,1870408.163,1704081.633,1704081.633,0,0,0,0,25,30,...,24100,32100,40000,49000,40000,49000,0.00,0.00,0,0
HydroRiver,0.000000e+00,0.000,0.000,0.000,0,0,0,0,75,75,...,11030,11030,11030,11030,11030,11030,0.00,0.00,0,0
NewHydroRiver,1.800000e+06,1800000.000,1800000.000,1800000.000,0,0,0,0,75,75,...,0,1200,0,1200,0,1200,0.00,0.00,0,0
Solar,9.450000e+05,822000.000,742000.000,742000.000,0,0,0,0,25,30,...,70300,234100,100000,350000,100000,350000,0.00,0.00,0,0


In [6]:
def tech_tab(Technos_df,year,prod_hyp):
    df_ret=pd.DataFrame()
    df_ret['energyCost']=Technos_df['energyCost_'+str(year)]+Technos_df['CO2_emissions']*d_carbon_tax[year]/1000
    df_ret['capacityCost']=(Technos_df['CAPEX_'+str(year)]\
                            +Technos_df['Decommissioning_'+str(year)]/(1+r)**Technos_df['Lifetime_'+str(year)])\
                            *(1-1/(1+r))/(1-1/(1+r)**Technos_df['Lifetime_'+str(year)])\
                            +Technos_df['OM_'+str(year)]
    df_ret['minCapacity']=Technos_df['minCapacity_'+prod_hyp+'_'+str(year)]
    df_ret['maxCapacity']=Technos_df['maxCapacity_'+prod_hyp+'_'+str(year)]
    for col in ["EnergyNbhourCap","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]:
        df_ret[col]=Technos_df[col]
    if prod_hyp=='100_enr':
        df_ret.loc['NewNuke','capacityCost']=(Technos_df.loc['NewNuke','CAPEX_2030']\
                            +Technos_df.loc['NewNuke','Decommissioning_'+str(year)]/(1+r)**37)\
                            *(1-1/(1+r))/(1-1/(1+r)**37)\
                            +Technos_df.loc['NewNuke','OM_'+str(year)]
    return df_ret

In [7]:
for year in [2030,2040,2050,2060]:
    for prod_hyp in ['nuclear_plus','nuclear_minus','100_enr']:
        df_tech=tech_tab(Technos_df,year,prod_hyp)
        df_tech.to_csv(Input_folder+"Technos_"+str(year)+"_"+prod_hyp+".csv",sep=";",decimal=",")

## Europe

In [9]:
Input_folder="Data/input/Prod_model/Prod_Europe/"
Technos_df=pd.read_csv(Input_folder+"Technos_Europe.csv",sep=";",decimal=",").set_index(["AREAS","TECHNOLOGIES"])
Technos_df

CAPEX_2030   CAPEX_2040   CAPEX_2050   CAPEX_2060  \
AREAS TECHNOLOGIES                                                          
DE    CCG             8.300000e+05   830000.000   830000.000   830000.000   
      Coal            1.600000e+06  1600000.000  1600000.000  1600000.000   
      WindOnShore     1.200000e+06  1050000.000   900000.000   900000.000   
      WindOffShore    2.036735e+06  1870408.163  1704081.633  1704081.633   
      HydroRiver      0.000000e+00        0.000        0.000        0.000   
...                            ...          ...          ...          ...   
ES    curtailment     0.000000e+00        0.000        0.000        0.000   
      HydroReservoir  0.000000e+00        0.000        0.000        0.000   
      Biomass         3.000000e+06  3000000.000  3000000.000  3000000.000   
      OldNuke         6.500000e+05   650000.000   650000.000   650000.000   
      NewNuke         1.157576e+07  5000000.000  4500000.000  4500000.000   

                      Decommissioning_2030  Decommissioning_2040  \
AREAS TECHNOLOGIES                                                 
DE    CCG                                0                     0   
      Coal                               0                     0   
      WindOnShore                        0                     0   
      WindOffShore                       0                     0   
      HydroRiver                         0                     0   
...                                    ...                   ...   
ES    curtailment                        0                     0   
      HydroReservoir                     0                     0   
      Biomass                            0                     0   
      OldNuke                       320000                320000   
      NewNuke                       200000                200000   

                      Decommissioning_2050  Decommissioning_2060  \
AREAS TECHNOLOGIES                                                 
DE    CCG                                0                     0   
      Coal                               0                     0   
      WindOnShore                        0                     0   
      WindOffShore                       0                     0   
      HydroRiver                         0                     0   
...                                    ...                   ...   
ES    curtailment                        0                     0   
      HydroReservoir                     0                     0   
      Biomass                            0                     0   
      OldNuke                       320000                320000   
      NewNuke                       200000                200000   

                      Lifetime_2030  Lifetime_2040  ...  minCapacity_2040  \
AREAS TECHNOLOGIES                                  ...                     
DE    CCG                        30             30  ...                 0   
      Coal                       40             40  ...                 0   
      WindOnShore                25             30  ...             73666   
      WindOffShore               25             30  ...             54418   
      HydroRiver                 75             75  ...              3800   
...                             ...            ...  ...               ...   
ES    curtailment                 1              1  ...                 0   
      HydroReservoir             75             75  ...             19187   
      Biomass                    30             30  ...                 0   
      OldNuke                    10             10  ...                 0   
      NewNuke                    60             60  ...                 0   

                      maxCapacity_2040  minCapacity_2050  maxCapacity_2050  \
AREAS TECHNOLOGIES                                                           
DE    CCG                        36250                 0                 0   
      Coal      

In [10]:
def tech_tab_E(Technos_df,year):
    df_ret=pd.DataFrame()
    df_ret['energyCost']=Technos_df['energyCost_'+str(year)]+Technos_df['CO2_emissions']*d_carbon_tax[year]/1000
    df_ret['capacityCost']=(Technos_df['CAPEX_'+str(year)]\
                            +Technos_df['Decommissioning_'+str(year)]/(1+r)**Technos_df['Lifetime_'+str(year)])\
                            *(1-1/(1+r))/(1-1/(1+r)**Technos_df['Lifetime_'+str(year)])\
                            +Technos_df['OM_'+str(year)]
    df_ret['minCapacity']=Technos_df['minCapacity_'+str(year)]
    df_ret['maxCapacity']=Technos_df['maxCapacity_'+str(year)]
    for col in ["EnergyNbhourCap","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]:
        df_ret[col]=Technos_df[col]
    return df_ret

In [13]:
for year in [2030,2040,2050,2060]:
    df_tech=tech_tab_E(Technos_df,year)
    df_tech.to_csv(Input_folder+"Technos_Europe_"+str(year)+".csv",sep=";",decimal=",")

# 2. Storage

In [14]:
Input_folder="Data/input/Prod_model/"
StockTechnos_df=pd.read_csv(Input_folder+"Stock_technos.csv",sep=";",decimal=",").set_index("STOCK_TECHNO")
StockTechnos_df

,CAPEX_power_in_2030,CAPEX_power_out_2030,CAPEX_energy_2030,CAPEX_power_in_2040,CAPEX_power_out_2040,CAPEX_energy_2040,CAPEX_power_in_2050,CAPEX_power_out_2050,CAPEX_energy_2050,CAPEX_power_in_2060,...,efficiency_out_2030,efficiency_in_2040,dissipation_2040,efficiency_out_2040,efficiency_in_2050,dissipation_2050,efficiency_out_2050,efficiency_in_2060,dissipation_2060,efficiency_out_2060
STOCK_TECHNO,,,,,,,,,,,,,,,,,,,,,
Battery,1.705385e+05,0,225282.5768,156868.054,0,172728.3345,1.732686e+05,0,141662.1695,1.732686e+05,...,0.95,0.95,0.000000,0.95,0.95,0.000000,0.95,0.95,0.000000,0.95
HydroStorage,1.325000e+06,0,0.0000,1325000.000,0,0.0000,1.325000e+06,0,0.0000,1.325000e+06,...,0.90,0.90,0.000000,0.90,0.90,0.000000,0.90,0.90,0.000000,0.90
PowertoH2toPower,1.350000e+06,1100000,0.0000,1080000.000,1100000,0.0000,8.000000e+05,1100000,0.0000,8.000000e+05,...,0.50,0.70,0.000012,0.58,0.70,0.000012,0.58,0.70,0.000012,0.58


In [15]:
def stock_tech_tab(StockTechnos_df,year):
    df_ret=pd.DataFrame()
    for char in ['in','out','energy']:
        char1=char
        if char in ['in','out']:
            char1='power_'+char1
        df_ret['storageCost_'+char1]=StockTechnos_df['CAPEX_'+char1+'_'+str(year)]\
                            *(1-1/(1+r))/(1-1/(1+r)**StockTechnos_df['Lifetime_'+char])\
                            +StockTechnos_df['OPEX_'+char+'_'+str(year)]
    df_ret['p_max_in']=StockTechnos_df['p_max_in_'+str(year)]
    df_ret['p_max_out']=StockTechnos_df['p_max_out_'+str(year)]
    df_ret['efficiency_in']=StockTechnos_df['efficiency_in_'+str(year)]
    df_ret['dissipation']=StockTechnos_df['dissipation_'+str(year)]
    df_ret['efficiency_out']=StockTechnos_df['efficiency_out_'+str(year)]
    for col in ["p_max_out_eq_p_max_in","h_max"]:
        df_ret[col]=StockTechnos_df[col]
    
    return df_ret

In [16]:
for year in [2030,2040,2050,2060]:
    df_stock_tech=stock_tech_tab(StockTechnos_df,year)
    df_stock_tech.to_csv(Input_folder+"Stock_technos_"+str(year)+".csv",sep=";",decimal=",")

## Europe

In [20]:
Input_folder="Data/input/Prod_model/Prod_Europe/"
StockTechnos_df=pd.read_csv(Input_folder+"Stock_technos_Europe.csv",sep=";",decimal=",").set_index(["AREAS","STOCK_TECHNO"])
StockTechnos_df

CAPEX_power_in_2030  CAPEX_power_out_2030  \
AREAS STOCK_TECHNO                                                  
FR    Battery                  1.705385e+05                     0   
      HydroStorage             1.325000e+06                     0   
      PowertoH2toPower         1.350000e+06               1100000   
BE    Battery                  1.732686e+05                     0   
      HydroStorage             1.325000e+06                     0   
      PowertoH2toPower         1.350000e+06               1100000   
CH    Battery                  0.000000e+00                     0   
      HydroStorage             1.325000e+06                     0   
      PowertoH2toPower         1.350000e+06               1100000   
DE    Battery                  0.000000e+00                     0   
      HydroStorage             1.325000e+06                     0   
      PowertoH2toPower         1.350000e+06               1100000   
ES    Battery                  0.000000e+00                     0   
      HydroStorage             1.325000e+06                     0   
      PowertoH2toPower         1.350000e+06               1100000   
GB    Battery                  0.000000e+00                     0   
      HydroStorage             1.325000e+06                     0   
      PowertoH2toPower         1.350000e+06               1100000   
IT    Battery                  0.000000e+00                     0   
      HydroStorage             1.325000e+06                     0   
      PowertoH2toPower         1.350000e+06               1100000   

                        CAPEX_energy_2030  CAPEX_power_in_2040  \
AREAS STOCK_TECHNO                                               
FR    Battery                 225282.5768           156868.054   
      HydroStorage                 0.0000          1325000.000   
      PowertoH2toPower             0.0000          1080000.000   
BE    Battery                 141662.1695                0.000   
      HydroStorage                 0.0000          1325000.000   
      PowertoH2toPower             0.0000          1080000.000   
CH    Battery                      0.0000                0.000   
      HydroStorage                 0.0000          1325000.000   
      PowertoH2toPower             0.0000          1080000.000   
DE    Battery                      0.0000                0.000   
      HydroStorage                 0.0000          1325000.000   
      PowertoH2toPower             0.0000          1080000.000   
ES    Battery                      0.0000                0.000   
      HydroStorage                 0.0000          1325000.000   
      PowertoH2toPower             0.0000          1080000.000   
GB    Battery                      0.0000                0.000   
      HydroStorage                 0.0000          1325000.000   
      PowertoH2toPower             0.0000          1080000.000   
IT    Battery                      0.0000                0.000   
      HydroStorage                 0.0000          1325000.000   
      PowertoH2toPower             0.0000          1080000.000   

                        CAPEX_power_out_2040  CAPEX_energy_2040  \
AREAS STOCK_TECHNO                                                
FR    Battery                              0        172728.3345   
      HydroStorage                         0             0.0000   
      PowertoH2toPower               1100000             0.0000   
BE    Battery                              0             0.0000   
      HydroStorage                         0             0.0000   
      PowertoH2toPower               1100000             0.0000   
CH    Battery                              0             0.0000   
      HydroStorage                         0             0.0000   
      PowertoH2toPower               1100000             0.0000   
DE    Battery                              0             0.0000   
      HydroStorage                         0             0.0000   
      PowertoH2toPower               1100000  

In [21]:
for year in [2030,2040,2050,2060]:
    df_stock_tech=stock_tech_tab(StockTechnos_df,year)
    df_stock_tech.to_csv(Input_folder+"Stock_technos_Europe_"+str(year)+".csv",sep=";",decimal=",")

# 3. Hydrogen

In [23]:
#Input_folder="Data/input/Conso_model/H2/"
#H2Technos_df=pd.read_csv(Input_folder+"ParametersH2.csv",sep=";",decimal=",").set_index("H2_TECHNO")
#H2Technos_df

In [24]:
#def H2_tech_tab(H2Technos_df,year):
    #df_ret=pd.DataFrame()
    #df_ret['capacityCostH2']=H2Technos_df['CAPEX_'+str(year)]\
                            #*(1-1/(1+r))/(1-1/(1+r)**H2Technos_df['Lifetime_'+str(year)])\
                            #+H2Technos_df['OPEX_'+str(year)]
    #df_ret['energyCostH2']=H2Technos_df['energyCostH2']
    #df_ret['efficiencyH2']=H2Technos_df['EfficiencyH2_'+str(year)]
    
    #return df_ret

In [25]:
#for year in [2030,2040,2050,2060]:
    #df_H2_tech=H2_tech_tab(H2Technos_df,year)
    #df_H2_tech.to_csv(Input_folder+"H2_technos_"+str(year)+".csv",sep=";",decimal=",")

# 4. Flexibility

In [22]:
Input_folder="Data/input/Conso_model/Flex/"
FlexTechnos_df=pd.read_csv(Input_folder+"FlexParameters.csv",sep=";",decimal=",").set_index("FLEX_CONSUM")
FlexTechnos_df

,CAPEX_2030,CAPEX_2040,CAPEX_2050,CAPEX_2060,Lifetime_2030,Lifetime_2040,Lifetime_2050,Lifetime_2060,OPEX_2030,OPEX_2040,OPEX_2050,OPEX_2060,flex_ratio,max_ratio,flex_type,labourcost
FLEX_CONSUM,,,,,,,,,,,,,,,,
Steel,2.772006e+06,2.772006e+06,2.772006e+06,2.772006e+06,20,20,20,20,83160.18613,83160.18613,83160.18613,83160.18613,0.9,1,week,53
EV,3.907226e+05,3.441756e+05,3.441756e+05,3.441756e+05,15,15,15,15,22681.81818,22681.81818,22681.81818,22681.81818,0.5,1,day_ev,0


In [23]:
def Flex_tech_tab(FlexTechnos_df,year):
    df_ret=pd.DataFrame()
    df_ret['LoadCost']=FlexTechnos_df['CAPEX_'+str(year)]\
                            *(1-1/(1+r))/(1-1/(1+r)**FlexTechnos_df['Lifetime_'+str(year)])\
                            +FlexTechnos_df['OPEX_'+str(year)]
    for col in ["flex_ratio","max_ratio","flex_type","labourcost"]:
        df_ret[col]=FlexTechnos_df[col]
    
    return df_ret

In [24]:
for year in [2030,2040,2050,2060]:
    df_flex_tech=Flex_tech_tab(FlexTechnos_df,year)
    df_flex_tech.to_csv(Input_folder+"Flex_"+str(year)+".csv",sep=";",decimal=",")

## Europe

In [28]:
Input_folder="Data/input/Conso_model/Conso_Europe/"
FlexTechnos_df=pd.read_csv(Input_folder+"Flex_Europe.csv",sep=";",decimal=",").set_index(["AREAS","FLEX_CONSUM"])
FlexTechnos_df

CAPEX_2030    CAPEX_2040    CAPEX_2050    CAPEX_2060  \
AREAS FLEX_CONSUM                                                           
FR    Steel        2.772006e+06  2.772006e+06  2.772006e+06  2.772006e+06   
      EV           3.907226e+05  3.441756e+05  3.441756e+05  3.441756e+05   
BE    Steel        2.772006e+06  2.772006e+06  2.772006e+06  2.772006e+06   
      EV           3.907226e+05  3.441756e+05  3.441756e+05  3.441756e+05   
CH    Steel        2.772006e+06  2.772006e+06  2.772006e+06  2.772006e+06   
      EV           3.907226e+05  3.441756e+05  3.441756e+05  3.441756e+05   
DE    Steel        2.772006e+06  2.772006e+06  2.772006e+06  2.772006e+06   
      EV           3.907226e+05  3.441756e+05  3.441756e+05  3.441756e+05   
ES    Steel        2.772006e+06  2.772006e+06  2.772006e+06  2.772006e+06   
      EV           3.907226e+05  3.441756e+05  3.441756e+05  3.441756e+05   
GB    Steel        2.772006e+06  2.772006e+06  2.772006e+06  2.772006e+06   
      EV           3.907226e+05  3.441756e+05  3.441756e+05  3.441756e+05   
IT    Steel        2.772006e+06  2.772006e+06  2.772006e+06  2.772006e+06   
      EV           3.907226e+05  3.441756e+05  3.441756e+05  3.441756e+05   

                   Lifetime_2030  Lifetime_2040  Lifetime_2050  Lifetime_2060  \
AREAS FLEX_CONSUM                                                               
FR    Steel                   20             20             20             20   
      EV                      15             15             15             15   
BE    Steel                   20             20             20             20   
      EV                      15             15             15             15   
CH    Steel                   20             20             20             20   
      EV                      15             15             15             15   
DE    Steel                   20             20             20             20   
      EV                      15             15             15             15   
ES    Steel                   20             20             20             20   
      EV                      15             15             15             15   
GB    Steel                   20             20             20             20   
      EV                      15             15             15             15   
IT    Steel                   20             20             20             20   
      EV                      15             15             15             15   

                     OPEX_2030    OPEX_2040    OPEX_2050    OPEX_2060  \
AREAS FLEX_CONSUM                                                       
FR    Steel        83160.18613  83160.18613  83160.18613  83160.18613   
      EV           22681.81818  22681.81818  22681.81818  22681.81818   
BE    Steel        83160.18613  83160.18613  83160.18613  83160.18613   
      EV           22681.81818  22681.81818  22681.81818  22681.81818   
CH    Steel        83160.18613  83160.18613  83160.18613  83160.18613   
      EV           22681.81818  22681.81818  22681.81818  22681.81818   
DE    Steel        83160.18613  83160.18613  83160.18613  83160.18613   
      EV           22681.81818  22681.81818  22681.81818  22681.81818   
ES    Steel        83160.18613  83160.18613  83160.18613  83160.18613   
      EV           22681.81818  22681.81818  22681.81818  22681.81818   
GB    Steel        83160.18613  83160.18613  83160.18613  83160.18613   
      EV           22681.81818  22681.81818  22681.81818  22681.81818   
IT    Steel        83160.18613  83160.18613  83160.18613  83160.18613   
      EV           22681.81818  22681.81818  22681.81818  22681.81818   

                   flex_ratio  max_ratio flex_type  labourcost  
AREAS FLEX_CONSUM                                               
FR    Steel               0.9          1      week          53  
      EV                  0.5          1    day_ev           0  
BE    Steel               0.9          1      week          53

In [29]:
for year in [2030,2040,2050,2060]:
    df_flex_tech=Flex_tech_tab(FlexTechnos_df,year)
    df_flex_tech.to_csv(Input_folder+"Flex_Europe_"+str(year)+".csv",sep=";",decimal=",")